<a href="https://colab.research.google.com/github/shrgtm/Automated-Meta-Data-Generation/blob/main/Automated_Meta_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber python-docx Pillow transformers -q
!apt-get install -y poppler-utils tesseract-ocr -qq
!pip install pytesseract -q



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.1 MB/s eta 0:00:00
Selecting previously unselected package poppler-utils.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import pytesseract
from PIL import Image
import docx
import pdfplumber
import io
import json
import base64

from transformers import pipeline
from google.colab import files


In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def extract_text(file_name, file_type):
    if file_type == ".pdf":
        with pdfplumber.open(file_name) as pdf:
            return "\n".join([page.extract_text() or "" for page in pdf.pages])

    elif file_type == ".docx":
        doc = docx.Document(file_name)
        return "\n".join([p.text for p in doc.paragraphs])

    elif file_type in [".png", ".jpg", ".jpeg"]:
        img = Image.open(file_name)
        return pytesseract.image_to_string(img)

    elif file_type == ".txt":
        with open(file_name, 'r', encoding="utf-8") as f:
            return f.read()

    else:
        return None


In [ ]:
def generate_metadata(text):
    short_text = text[:3000]  # Reduce input for summarizer
    summary = summarizer(short_text, max_length=120, min_length=30, do_sample=False)[0]['summary_text']
    keywords = list(set(summary.lower().split()))

    metadata = {
        "title": summary.split('.')[0],
        "summary": summary,
        "keywords": keywords[:10],
        "author": "Unknown",
        "date": "Unknown"
    }
    return metadata


In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"📁 Uploaded: {file_name}")





Saving software_engineer_resume.pdf to software_engineer_resume.pdf
📁 Uploaded: software_engineer_resume.pdf


In [ ]:
import os

if 'file_name' in globals():
    if '.' in file_name:
        name_parts = os.path.splitext(file_name)
        if len(name_parts) == 2:
            _, ext = name_parts
            text = extract_text(file_name, ext.lower())

            if not text or len(text.strip()) < 20:
                print("❌ Could not extract enough text. Try another file.")
            else:
                metadata = generate_metadata(text)
                print("✅ Metadata Generated Successfully:\n")
                print(json.dumps(metadata, indent=4))

                # Save and download metadata
                with open("metadata.json", "w") as f:
                    json.dump(metadata, f, indent=4)

                files.download("metadata.json")
        else:
            print("❌ File name does not have a valid extension.")
    else:
        print("❌ File does not seem to have an extension.")
else:
    print("❌ File was not uploaded correctly. Please re-upload.")



✅ Metadata Generated Successfully:

{
    "title": "John Smith is a Software Engineer with 5+ years of experience in designing, developing, and implementing scalable web applications and solutions",
    "summary": "John Smith is a Software Engineer with 5+ years of experience in designing, developing, and implementing scalable web applications and solutions. He is skilled in full-stack development, cloud technologies, and Agile methodologies. John Smith is currently working for Tech Solutions Inc. in San Francisco.",
    "keywords": [
        "a",
        "solutions",
        "agile",
        "development,",
        "inc.",
        "designing,",
        "for",
        "experience",
        "of",
        "san"
    ],
    "author": "Unknown",
    "date": "Unknown"
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>